In [ ]:
# 1. 알고(구현): 백준 16926 (배열 돌리기 1) - https://www.acmicpc.net/problem/16926
import sys
from collections import deque

from io import StringIO
sys.stdin = StringIO('''5 4 7
1 2 3 4
7 8 9 10
13 14 15 16
19 20 21 22
25 26 27 28''')

input = sys.stdin.readline


def solve():
    N, M, R = map(int, input().split())

    board = [list(input().split()) for _ in range(N)]
    directions = [(+1, 0), (0, +1), (-1, 0), (0, -1)]

    for layer in range(min(N, M) // 2):
        sr, er = 0 + layer, N - 1 - layer   # 경계 값
        sc, ec = 0 + layer, M - 1 - layer

        r, c = sr, sc
        row_col = list()
        value_dq = deque()

        d = 0
        while True:
            dr, dc = directions[d]
            nr, nc = r + dr, c + dc
            if sr <= nr <= er and sc <= nc <= ec:
                row_col.append((nr, nc))
                value_dq.append(board[nr][nc])

                r, c = nr, nc
                if (r, c) == (sr, sc):              # 시작위치 오면 탈출
                    break
            else:
                d = (d + 1) % 4

        value_dq.rotate(R)                           # 회전 시킨 값
        for new_r, new_c in row_col:
            board[new_r][new_c] = value_dq.popleft()

    for row in board:
        print(*row)


solve()

28 27 26 25
22 9 15 19
16 8 21 13
10 14 20 7
4 3 2 1


In [39]:
# 2. 알고(수학): 백준 1748 (수 이어 쓰기 1) - https://www.acmicpc.net/problem/1748
import sys
from math import log10

from io import StringIO
sys.stdin = StringIO('''120''')

input = sys.stdin.readline


def solve():
    N = int(input())


    digit_cnt = int(log10(N))
    N = N - 10**digit_cnt
    cnt_val = N * (digit_cnt + 1) + 1
    print(cnt_val)
    while digit_cnt > 0:
        print(f"{digit_cnt=}", (10**digit_cnt - 10*(digit_cnt-1))*digit_cnt)
        cnt_val += (10**digit_cnt - 10*(digit_cnt-1))*digit_cnt
        digit_cnt -= 1

    print(cnt_val)

solve()


61
digit_cnt=2 180
digit_cnt=1 10
251


In [ ]:
# 3. 알고(Map): 백준 5568 (카드 놓기) - https://www.acmicpc.net/problem/5568
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


: 

In [ ]:
# 4. 알고(그리디): 백준 1092 (배) - https://www.acmicpc.net/problem/1092
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


```sql
-- 5. SQL(Lv.2): 조건에 맞는 도서와 저자 리스트 출력하기 - https://school.programmers.co.kr/learn/courses/30/lessons/144854
```

In [ ]:
# 1. 알고(시뮬): 백준 16236 (아기 상어) - https://www.acmicpc.net/problem/16236
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


In [ ]:
# 2. 알고(DP): 백준 11055 (가장 큰 증가하는 부분 수열) - https://www.acmicpc.net/problem/11055
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


In [ ]:
# 3. 알고(그래프): 백준 17396 (백도어) - https://www.acmicpc.net/problem/17396
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


In [ ]:
# 4. 알고(백트래킹): 백준 1405 (미친 로봇) - https://www.acmicpc.net/problem/1405
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


```sql
-- 5. SQL(Lv.3): 없어진 기록 찾기 - https://school.programmers.co.kr/learn/courses/30/lessons/59042
```